Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

# 1. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
try:
    data = pd.read_csv(r'C:\Users\kuzmi\Downloads\insurance.csv')
except:
    data = pd.read_csv(r'/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


<div class="alert alert-success">
<b>Комментарий ревьюера ✔️:</b> Супер, все хорошо </div>

# 2. Умножение матриц


Oбозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Нам необхoдимо доказать что $$a = a_p$$
следовательно должно получится $$Xw = PXw_p$$

Формула обучения:

$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
$$
w = (X^T X)^{-1} X^T y
$$
\


$w' = (XP^T XP)^{-1} XP^T y$

 $(XP)^T$ = $(P^T X^T X P)^{-1}$
 \
 $(ABC)^{-1} = (BC)^{-1}A^{-1} = C^{-1}B^{-1}A^{-1}$
 \
 $P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^Ty = (X^TX)^{-1}P^TX^Ty $ 
 \
 $w' = P^{-1}w$
 \
 $a' = X'P^{-1}w$
 \
 $a' = XPP^{-1}$
 \
 $a' = Xw$
 \
 $a' = a$
 



**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится

**Обоснование:** Приведено выше

# 3. Алгоритм преобразования

**Алгоритм**

В качестве алгоритма преобразования примем домножение обучающих признаков X на случайную обратимую матрицу P размером 4х4. Создадим модель линейной регрессии до преобразования и посчитаем метрику R2. Далее умножим исходные признаки на обратимую матрицу и на основе полученных значений вновь посчитаем метрику R2. Метрики R2 по итогу должны быть равны.

**Обоснование**

Как было показано в предыдущем пункте, домножение признаков на какую-либо обратимую матрицу не приводит к изменению предсказаний, поэтому результаты R2 должны оказаться равны

#### Разделение данных на целевую и обучающую выборки

In [5]:
features=data.drop(['Страховые выплаты'],axis=1)
target=data['Страховые выплаты']

#### Создаем выборки обуч и тест

In [6]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

#### Модель с исходными признаками без преобразования

In [7]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_origin_data = r2_score(target_test, model.predict(features_test))
print("R2 =", R2_LR_origin_data)

R2 = 0.43522757127026546


#### Модель с отмасштабированными признаками без преобразования

In [8]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_origin_data_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_LR_origin_data_scaled)

R2 = 0.4352275712702668


### Вывод
Качество модели практически не меняется

## Иследование качества модели c преобразованием

In [9]:
# Создадим рандомную матрицу-ключ
n = features.shape[1]
crypto_matrix = np.random.randint(1,10, (n,n))
#Признаки защищенные
crypto_features = features
crypted_features = crypto_features @ crypto_matrix
np.linalg.inv(crypto_matrix)

array([[ 0.30487805, -0.34146341, -0.37804878,  0.62195122],
       [ 0.17073171,  0.04878049, -0.06504065, -0.06504065],
       [-0.1402439 , -0.18292683,  0.16056911,  0.16056911],
       [-0.29268293,  0.48780488,  0.3495935 , -0.6504065 ]])

In [10]:
display(features.head())
display(crypted_features.head())
crypto_matrix

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


,0,1,2,3
0,198779.0,396851.0,396890.0,248378.0
1,152422.0,304047.0,304096.0,190421.0
2,84261.0,168029.0,168058.0,105261.0
3,167005.0,333623.0,333650.0,208703.0
4,104654.0,208837.0,208860.0,130754.0


array([[2, 9, 4, 2],
       [9, 1, 2, 9],
       [4, 8, 8, 5],
       [8, 1, 4, 7]])

### Вывод
Видно, что оригинальные данные заменились на совершенно другие цифры.

## Проверим качество модели

In [11]:
features_train, features_test, target_train, target_test = train_test_split(
    crypted_features, target, test_size=0.25, random_state=12345)

In [12]:
# Модель с исходными признаками без преобразования
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_cipher_data = r2_score(target_test, model.predict(features_test))
print("R2 =", R2_LR_cipher_data)

R2 = 0.4352275712703343


In [13]:
# Модель с отмасштабированными признаками без преобразования
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_cipher_data_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_LR_cipher_data_scaled)

R2 = 0.43522757127035416


# Вывод
Качество линейной регресии не изменилось от использования исxодной матрицы и исходной матрицы, умноженную на обратимую. Это мы смогли доказать по формуле, а также практическим путем (была исследована метрика R2).